In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "YOUR-API-KEY",
    model_name="llama-3.1-8b-instant",
)
response = llm.invoke("Who is Newton?")
print(response.content)

Sir Isaac Newton (1643-1727) was a renowned English mathematician, physicist, astronomer, and scientist who made significant contributions to various fields, including mathematics, optics, and physics. He is widely recognized as one of the most influential scientists in history.

Newton's most notable contributions include:

1. **Laws of Motion**: Newton formulated three laws of motion that describe how objects move and respond to forces. These laws are still widely used today in physics and engineering.
2. **Universal Gravitation**: Newton's law of universal gravitation states that every point mass attracts every other point mass by a force acting along the line intersecting both points. This law explained the motion of planets, moons, and comets.
3. **Calculus**: Newton, along with German mathematician Gottfried Wilhelm Leibniz, developed calculus, a branch of mathematics that deals with rates of change and accumulation.
4. **Optics**: Newton's work on optics led to the development o

In [10]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.siemens-healthineers.com/en_US/searchJobs/JobDetail/488469?source=LinkedIn&sourceType=PREMIUM_POST_SITE")
page_data = loader.load().pop().page_content
print(page_data)















        Technical Intern - Job detail | Siemens Healthineers AG CareersMarketplace
    






 









                            Skip to content
                        

                            Skip to footer
                        

















                Jobs Home
            





                All Jobs
            





                AI Recommendations
            





                FAQs & Support
            








                                                                    English
                                                            


DeutschEnglishEspañolFrançaisNederlandsPortugueseČestina中文 (简体)中文 (繁体)日本語







                Login
            





 







                Jobs Home
            





                All Jobs
            





                AI Recommendations
            





                FAQs & Support
            




 









                                Back to search                    

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

{
  "role": "Technical Intern",
  "experience": "Student (Not Yet Graduated)",
  "skills": [
    "Software design",
    "Coding and quality standards",
    "Test-Driven Development / Behavior-Driven Development",
    "Extreme programming methods",
    "OOPS, design principles & patterns, inter-process communication, multi-threading, and data-structures"
  ],
  "description": "Join us in our fight against the world's most dangerous diseases and in enabling access to care for more people around the world! At Siemens Healthineers, we pioneer breakthroughs in healthcare. For everyone. Everywhere. We provide a flexible and dynamic environment in which you can stretch yourself personally and professionally. "
}


In [12]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Technical Intern',
 'experience': 'Student (Not Yet Graduated)',
 'skills': ['Software design',
  'Coding and quality standards',
  'Test-Driven Development / Behavior-Driven Development',
  'Extreme programming methods',
  'OOPS, design principles & patterns, inter-process communication, multi-threading, and data-structures'],
 'description': "Join us in our fight against the world's most dangerous diseases and in enabling access to care for more people around the world! At Siemens Healthineers, we pioneer breakthroughs in healthcare. For everyone. Everywhere. We provide a flexible and dynamic environment in which you can stretch yourself personally and professionally. "}

In [13]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [16]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])


In [21]:
links = collection.query(query_texts=job["skills"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [23]:
job

{'role': 'Technical Intern',
 'experience': 'Student (Not Yet Graduated)',
 'skills': ['Software design',
  'Coding and quality standards',
  'Test-Driven Development / Behavior-Driven Development',
  'Extreme programming methods',
  'OOPS, design principles & patterns, inter-process communication, multi-threading, and data-structures'],
 'description': "Join us in our fight against the world's most dangerous diseases and in enabling access to care for more people around the world! At Siemens Healthineers, we pioneer breakthroughs in healthcare. For everyone. Everywhere. We provide a flexible and dynamic environment in which you can stretch yourself personally and professionally. "}

In [20]:
job = json_res
job["skills"]

['Software design',
 'Coding and quality standards',
 'Test-Driven Development / Behavior-Driven Development',
 'Extreme programming methods',
 'OOPS, design principles & patterns, inter-process communication, multi-threading, and data-structures']

In [22]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Your Potential with AtliQ - Expert AI & Software Consulting

Dear [Hiring Manager's Name],

I came across the job description for a Technical Intern at Siemens Healthineers and was impressed by the exciting opportunity to join a pioneering team in the fight against the world's most dangerous diseases. As a Business Development Executive at AtliQ, I'd like to introduce you to our AI & Software Consulting company that can help you achieve your goals.

AtliQ has a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise spans a wide range of technologies, including software design, coding, test-driven development, extreme programming methods, and more.

I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

- **Android Development**: We've successfully developed scalable and efficient Android applications 